## Setup RPC & deployer

In [ ]:
from dotenv import load_dotenv
import os
import boa
from eth_account import Account
from web3 import Web3
import logging
import subprocess
import time

load_dotenv()
chain_handle = "base-sepolia"

PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")

RPCs = {
    "sepolia": "https://eth-sepolia.public.blastapi.io",
    "base-sepolia": "https://sepolia.base.org",
    "optimism-sepolia": "https://sepolia.optimism.io",
    "arbitrum-sepolia": "https://sepolia-rollup.arbitrum.io/rpc",
}

state_dict = {}
# RPC_URL = RPCs[chain_handle]

deployer = Account.from_key(PRIVATE_KEY)

for key in RPCs.keys():
    state_dict[key] = {}
    state_dict[key]["rpc"] = RPCs[key]
    boa.set_network_env(RPCs[key])
    boa.env.add_account(deployer)
    state_dict[key]["boa"] = boa.env  # oops
    state_dict[key]["w3"] = Web3(Web3.HTTPProvider(RPCs[key]))

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        print(f"Deploying with {boa.env.eoa} on {key}, id {boa.env.evm.patch.chain_id}")
        print(
            f"Chain balance is {state_dict[key]['w3'].eth.get_balance(boa.env.eoa)/1e18 :.3f} ETH"
        )

## Deploy block oracles and main view contract

In [ ]:
main_chain = "sepolia"

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]["boa"]):
        if key == main_chain:
            contract_deployer = boa.load_partial("../contracts/MainnetBlockView.vy")
            contract = contract_deployer()
            print(f"Block view deployed at {contract.address} on {key}")
        else:
            contract_deployer = boa.load_partial("../contracts/BlockOracle.vy")
            contract = contract_deployer(1)  # commit threshold
            print(f"Block oracle deployed at {contract.address} on {key}")
        state_dict[key]["oracle"] = contract

## Setup LZ

In [ ]:
from LZDeployments import LZDeployments

lz = LZDeployments()
for chain in state_dict:
    metadata = lz.get_chain_metadata(chain)["metadata"]
    dvn_data = lz.get_chain_metadata(chain)

    # Update state dict
    state_dict[chain].update(
        {
            "eid": metadata["eid"],
            "endpoint": metadata["endpointV2"],
            "send_lib": metadata.get("sendUln302", "unavailable"),
            "receive_lib": metadata.get("receiveUln302", "unavailable"),
            "read_lib": metadata.get("readLib1002", "unavailable"),
            "dvns": dvn_data["dvns"],
        }
    )

    # Print info
    print(f"LZ details for {chain}:")
    print(f"Chain eID: {metadata['eid']}\nEndpoint address: {metadata['endpointV2']}")
    print(f"DVNs: {len(dvn_data['dvns'])}, Read DVNs: {len(dvn_data['dvns_lzread'])}")
    print(
        f"Send lib: {state_dict[chain]['send_lib']}\n"
        f"Receive lib: {state_dict[chain]['receive_lib']}\n"
        f"Read lib: {state_dict[chain]['read_lib']}\n---"
    )

# I. Deployment

In [ ]:
# Deploy LZBlockRelay on each chain

for chain in state_dict:
    if chain == main_chain:
        continue
    with boa.swap_env(state_dict[chain]["boa"]):
        read_channel = 4294967295 if not state_dict[chain]["read_lib"] == "unavailable" else 0
        contract_deployer = boa.load_partial("../contracts/messengers/LZBlockRelay.vy")
        contract = contract_deployer(
            state_dict[chain]["endpoint"],  # endpoint on the chain
            200_000,  # default gas limit
            read_channel,  # read channel
        )
        print(f"LZ Messenger deployed at {contract.address} on {chain}")
        state_dict[chain]["messenger"] = contract

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [ ]:
def get_vyper_abi(filepath):
    command = ["vyper", filepath, "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")

ABI_RELAY = get_vyper_abi("../contracts/messengers/LZBlockRelay.vy")
ABI_ORACLE = get_vyper_abi("../contracts/BlockOracle.vy")
for key in state_dict.keys():
    if key == main_chain:
        continue
    state_dict[key]["block_relay_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["messenger"].address, abi=ABI_RELAY
    )
    state_dict[key]["oracle_w3"] = state_dict[key]["w3"].eth.contract(
        address=state_dict[key]["oracle"].address, abi=ABI_ORACLE
    )
account = Web3().eth.account.from_key(deployer.key)


def send_tx_single(w3, func, acc, value=0):
    tx = func.build_transaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "value": value,
        }
    )
    tx["gas"] = int(5 * w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(w3, func, acc, value=0):
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value)
            success = True
        except Exception as e:
            if "replacement transaction underpriced" in str(e) or "nonce too low" in str(e):
                print(str(e), "Retrying...")
                success = False
                time.sleep(0.5)
            else:
                raise e
    return tx_hash

## Prepare Libs

### send/receive (uln302 only)

In [ ]:
# send lib
for key in state_dict.keys():
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        func = state_dict[key]["block_relay_w3"].functions.set_lz_send_lib(
            state_dict[key]["eid"], state_dict[key]["send_lib"]
        )
        tx_hash = send_tx(state_dict[key]["w3"], func, account)
        print(f"Added send lib: {tx_hash.hex()} on {key}")

# receive lib
for key in state_dict.keys():
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        func = state_dict[key]["block_relay_w3"].functions.set_lz_receive_lib(
            state_dict[key]["eid"], state_dict[key]["receive_lib"]
        )
        tx_hash = send_tx(state_dict[key]["w3"], func, account)
        print(f"Added receive lib: {tx_hash.hex()} on {key}")

## read libs (if supported)

In [ ]:
from lzreadabi import lzreadlib_abi

for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        continue
    with boa.swap_env(state_dict[key]["boa"]):
        w3 = state_dict[key]["w3"]
        block_relay_w3 = state_dict[key]["block_relay_w3"]
        lzreadlib = w3.eth.contract(address=state_dict[key]["read_lib"], abi=lzreadlib_abi)
        print(f"Read lib type: {lzreadlib.functions.messageLibType().call()}")
        print(f"Read lib version: {lzreadlib.functions.version().call()}")

        eid_check = [4294967295, 4294967294]
        supported_eid = None
        for eid in eid_check:
            support = lzreadlib.functions.isSupportedEid(eid).call()
            print(f"Supports {eid}: {support}")
            if support and not supported_eid:
                supported_eid = eid
        ## read lib
        read_channel = block_relay_w3.functions.LZ_READ_CHANNEL().call()
        if supported_eid and read_channel != supported_eid:
            print(f"Setting read channel to {supported_eid}")
            func = block_relay_w3.functions.set_lz_read_channel(supported_eid)
            tx_hash = send_tx(w3, func, account)
            print(f"Set read channel: {tx_hash.hex()}")

        # read direction
        read_channel = block_relay_w3.functions.LZ_READ_CHANNEL().call()
        print(f"Read Channel: {read_channel}")
        func = block_relay_w3.functions.set_lz_send_lib(read_channel, state_dict[key]["read_lib"])
        tx_hash3 = send_tx(w3, func, account)
        print(f"Added read lib: {tx_hash3.hex()}")

        # # receive direction
        func = block_relay_w3.functions.set_lz_receive_lib(
            read_channel, state_dict[key]["read_lib"]
        )
        tx_hash4 = send_tx(w3, func, account)
        print(f"Added receive lib: {tx_hash4.hex()}")

## Set up DVNs

In [ ]:
# # first set the delegate
# # func = contract_w3.functions.set_lz_delegate(account.address)
# # tx_hash = send_tx(func, account)
# # logging.info(f"Added self as delegate: {tx_hash.hex()}")
# def checksum(address):
#     return Web3.to_checksum_address(address)


# CONFIG_TYPE_READ = 1
# CONFIG_TYPE_ULN = 2
# # list all dvns
# print("Listing all dvns")
# for dvn in dvns_all:
#     print(dvn)

# oapp = contract.address
# confirmations = 1
# required_dvns = [dvn["address"] for dvn in dvns_all if dvn["id"] == "layerzero-labs"]
# required_dvns = required_dvns[0:1]  # of all eid dvns we pick first one with layerzero id
# optional_dvns = [dvn["address"] for dvn in dvns_all if dvn["address"] not in required_dvns]
# # all other dvs are optional, with a threshold of 2 (or less is there is not enough)
# optional_dvn_threshold = min(2, len(optional_dvns))  # 2 or 1

# # sort dvns alphabetically (important, otherwise lz will fail!)
# required_dvns.sort()
# optional_dvns.sort()
# required_dvns = [checksum(dvn) for dvn in required_dvns]
# optional_dvns = [checksum(dvn) for dvn in optional_dvns]

# print("DVNs for send/receive:")
# print(f"Required: {required_dvns}")
# print(f"Optional: {optional_dvns}")
# print(f"Optional threshold: {optional_dvn_threshold}")
# for lib in [SEND_LIB, RECEIVE_LIB]:
#     func = contract_w3.functions.set_lz_uln_config(
#         LZ_EID,
#         oapp,
#         lib,
#         CONFIG_TYPE_ULN,
#         confirmations,
#         required_dvns,
#         optional_dvns,
#         optional_dvn_threshold,
#     )
#     tx_hash = send_tx(func, account)
#     print(f"Added ULN config: {tx_hash.hex()}")
#     # break

# required_dvns = [
#     dvn["address"]
#     for dvn in dvns_all
#     if dvn["id"] == "layerzero-labs" and dvn.get("lzReadCompatible", False)
# ]
# optional_dvns = [
#     dvn["address"]
#     for dvn in dvns_all
#     if dvn["address"] not in required_dvns and dvn.get("lzReadCompatible", False)
# ]
# optional_dvn_threshold = min(2, len(optional_dvns))  # 2 or 1 or 0
# print("DVNs for read:")
# print(f"Required: {required_dvns}")
# print(f"Optional: {optional_dvns}")
# print(f"Optional threshold: {optional_dvn_threshold}")
# required_dvns.sort()
# optional_dvns.sort()
# required_dvns = [checksum(dvn) for dvn in required_dvns]
# optional_dvns = [checksum(dvn) for dvn in optional_dvns]
# # required_dvns = []
# for lib in [READ_LIB]:
#     func = contract_w3.functions.set_lz_uln_config(
#         read_channel,
#         oapp,
#         lib,
#         CONFIG_TYPE_READ,
#         confirmations,
#         required_dvns,
#         optional_dvns,
#         optional_dvn_threshold,
#     )
#     tx_hash = send_tx(func, account)
#     print(f"Added ULN-Read config: {tx_hash.hex()}")

## LZ Block Relay

In [ ]:
# add block oracles everywhere
for key in state_dict.keys():
    if key == main_chain:
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]
    if contract_w3.functions.block_oracle().call() == state_dict[key]["oracle"].address:
        logging.info(f"Skipping {key} - already set")
        continue
    func = contract_w3.functions.set_block_oracle(state_dict[key]["oracle"].address)
    tx_hash = send_tx(state_dict[key]["w3"], func, account)
    logging.info(f"Set block oracle tx: {tx_hash.hex()} on {key}")

# now add relay as committer to block oracle
for key in state_dict.keys():
    if key == main_chain:
        continue
    if (
        state_dict[key]["oracle_w3"]
        .functions.is_committer(state_dict[key]["block_relay_w3"].address)
        .call()
    ):
        logging.info(f"Skipping {key} - already a committer")
        continue
    func = state_dict[key]["oracle_w3"].functions.add_committer(
        state_dict[key]["block_relay_w3"].address
    )
    tx_hash = send_tx(state_dict[key]["w3"], func, account)
    logging.info(f"Add committer tx: {tx_hash.hex()} on {key}")

In [ ]:
# set self as read peer (so read requests can be sent)
for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]
    oapp_address = state_dict[key]["messenger"].address  # we only request read ourselves
    LZ_READ_CHANNEL = contract_w3.functions.LZ_READ_CHANNEL().call()
    func = contract_w3.functions.set_peer(LZ_READ_CHANNEL, oapp_address)
    tx_hash = send_tx(state_dict[key]["w3"], func, account)
    logging.info(f"Set peer tx: {tx_hash.hex()} on {key}")

In [ ]:
# add mainnet view contract on read chains (as read destination)
for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]
    func = contract_w3.functions.set_read_config(
        True, state_dict[main_chain]["eid"], state_dict[main_chain]["oracle"].address
    )
    tx_hash = send_tx(state_dict[key]["w3"], func, account)
    logging.info(f"Set read config tx: {tx_hash.hex()} on {key}")

### Simple reads without broadcast

In [ ]:
main_block = state_dict[main_chain]["w3"].eth.block_number
print(f"Current block number on {main_chain}: {main_block}")
for key in state_dict.keys():
    if key == main_chain or state_dict[key]["read_lib"] == "unavailable":
        logging.info(f"Skipping {key}")
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]
    # First quote read fee
    fee = contract_w3.functions.quote_read_fee().call()
    logging.info(f"Read fee: {fee} on {key}")

    # Then request read
    func = contract_w3.functions.request_block_hash([], [], 0, 500_000)
    tx_hash = send_tx(state_dict[key]["w3"], func, account, 3 * fee)
    logging.info(f"Tx: {tx_hash.hex()} on {key}")

In [ ]:
# Check oracle data (wait until lz message propagates)
for key in state_dict.keys():
    if key == main_chain:
        continue
    number = state_dict[key]["oracle_w3"].functions.last_confirmed_block_number().call()
    block_hash = state_dict[key]["oracle_w3"].functions.block_hash(number).call()
    logging.info(f"Last confirmed block on {key}: {number}")
    logging.info(f"Block hash: {block_hash.hex()}")
    logging.info(f"Number difference: {main_block - number}")

### Now reads with broadcast

In [ ]:
# first add all LZ read contracts as peers to all chains
for read_key in state_dict.keys():  # cycle through all read chains
    if read_key == main_chain or state_dict[read_key]["read_lib"] == "unavailable":
        continue
    read_eid = state_dict[read_key]["eid"]
    read_oapp = state_dict[read_key]["messenger"].address
    contract_w3_read = state_dict[read_key]["block_relay_w3"]
    for key in state_dict.keys():  # cycle through all chains and
        if key == main_chain:
            continue
        contract_w3 = state_dict[key]["block_relay_w3"]
        # add read as peer
        func = contract_w3.functions.set_peer(read_eid, read_oapp)
        tx_hash = send_tx(state_dict[key]["w3"], func, account)
        logging.info(
            f"{key} - set peer {read_oapp} (eid {read_eid}) for messenger {contract_w3.address}: {tx_hash.hex()}"
        )
        # add chain to read broadcast targets
        if key != read_key:  # don't broadcast to self
            func = contract_w3_read.functions.add_broadcast_target(
                state_dict[key]["eid"], state_dict[key]["messenger"].address
            )
            tx_hash = send_tx(state_dict[read_key]["w3"], func, account)
            logging.info(f"{read_key} - added {key} to broadcast targets: {tx_hash.hex()}")

In [ ]:
# we can quote read fee now
# let's pick one of the read chains and quote fee
broadcaster = "base-sepolia"
broadcaster_eid = state_dict[broadcaster]["eid"]
broadcaster_w3 = state_dict[broadcaster]["block_relay_w3"]
receive_eids = [
    state_dict[key]["eid"] for key in state_dict.keys() if key not in [broadcaster, main_chain]
]
broadcast_fees = broadcaster_w3.functions.quote_broadcast_fees(receive_eids).call()
logging.info(f"LZSend fees: {broadcast_fees} on {broadcaster}")

BROADCAST_GAS = 2_000_000
read_fee_with_broadcast = broadcaster_w3.functions.quote_read_fee(
    0, BROADCAST_GAS, sum(broadcast_fees)
).call()
logging.info(f"LZRead fee with broadcast: {read_fee_with_broadcast} on {broadcaster}")

# magic broadcast call
func = broadcaster_w3.functions.request_block_hash(receive_eids, broadcast_fees, 0, BROADCAST_GAS)
tx_hash = send_tx(state_dict[broadcaster]["w3"], func, account, read_fee_with_broadcast)
logging.info(f"Tx: {tx_hash.hex()} on {broadcaster}")

## 1. LZ Send & Receive

In [ ]:
# add self as peer
for key in state_dict.keys():
    if key == main_chain:
        continue
    contract_w3 = state_dict[key]["block_relay_w3"]
    eid = state_dict[key]["eid"]
    oapp_address = state_dict[key]["messenger"].address
    func = contract_w3.functions.set_peer(eid, oapp_address)
    tx_hash = send_tx(state_dict[key]["w3"], func, account)
    logging.info(f"Added self as peer tx: {tx_hash.hex()} on {key}")

In [ ]:
# send messages
for key in state_dict.keys():
    if key == main_chain:
        continue

    contract_w3 = state_dict[key]["block_relay_w3"]
    gas_limit = 500_000
    value = int(0.000 * 10**18)
    msg = "170475436437825620930817601234267694881687829390282260281137596999800372275961"
    eid = state_dict[key]["eid"]
    oapp_address = state_dict[key]["messenger"].address

    fee = contract_w3.functions.quote_message_fee(
        eid, oapp_address, msg, _gas_limit=gas_limit, _value=value
    ).call()
    logging.info(f"Fee: {fee} on {key}")

    func = contract_w3.functions.send_message(
        _dst_eid=eid, _receiver=oapp_address, _message=msg, _gas_limit=gas_limit, _value=value
    )
    tx_hash = send_tx(state_dict[key]["w3"], func, account, 2 * fee)
    logging.info(f"Tx: {tx_hash.hex()}")
    # after we sent the message we can check it on testnet.layerzeroscan.com (use address or txid)
    # shortly after, it should be pinged back (we send to ourselves, and event be fired)

## 2. LzRead

In [ ]:
# # test calldata
# import requests

# print(calldata.hex())

# rpc_endpoint = "https://sepolia.base.org"
# # perform eth_call request to contract using calldata
# r = requests.post(
#     rpc_endpoint,
#     json={
#         "jsonrpc": "2.0",
#         "method": "eth_call",
#         "params": [{"to": CONTRACT_ADDRESS, "data": "0x" + calldata.hex()}, "latest"],
#         "id": 1,
#     },
# )
# response = r.json()["result"]
# # print(r.json()['result'])
# print(response)
# contract_code = """
# @external
# @view
# def message_as_string(_message: Bytes[128]) -> String[128]:
#     message: String[128] = convert(_message, String[128])
#     return message
# """
# with boa.swap_env(boa.Env()):
#     tmp_contract = boa.loads(contract_code)

#     res = tmp_contract.message_as_string(bytes.fromhex(response[2:]))
#     print(res)

In [ ]:
contract_code = """
@external
@view
def foo():
    val_1: uint128 = 123
    val_2: uint128 = 0
    val1_b16: bytes16 = convert(1, bytes16)
    val2_b16: bytes16 = convert(2, bytes16)
    # val_b32: bytes32 = convert(val_uint, bytes32)
    # a16: Bytes[16] = empty(Bytes[16])
    # a32: Bytes[32] = empty(Bytes[32])
    val_B32: Bytes[32] = concat(val1_b16, empty(bytes16))
    # # a1: bytes16 = convert(1, bytes16)
    print(val_B32)
"""
with boa.swap_env(boa.Env()):
    tmp_contract = boa.loads(contract_code)

    res = tmp_contract.foo()
    # print(res)